In [1]:
#!/usr/bin/python

"""

Needleman-Wunsch Aligner
Bioengineering 131/231, Fall 2018

Command-line script to read in the contents of a multi-FASTA containing two
sequences and a score matrix, perform a global alignment, and print the
resulting alignment matrix and optimal alignment to STDOUT.

"""

import os
import sys

class NWAligner:
    def __init__(self, score_matrix_fname):
        self.score_matrix, self.gap_penalty = self.load_score_matrix(score_matrix_fname)

    @staticmethod
    def load_score_matrix(fname):
        """
        Input: (String) A path to a scoring matrix file.
        Output: (Dictionary) A nested dictionary where keys are strings
                and elements are scores as integers.
    
        Example:
    
        >>> matrix, gap_penalty = NWAligner.load_score_matrix('/home/bioe131/BLOSUM62')
        >>> matrix['A']['A']
        4
        >>> matrix['W']['W']
        11
        >>> gap_penalty
        -4

        """

        score_matrix = {}
        gap_penalty = None
        keys = []

        with open(fname) as fp:
            for line_num, line in enumerate(fp):
                # ignore comments in matrix file
                if line.startswith("#"):
                    continue
                # Parse matrix file line-by-line and load into nested dictionaries.
                elif line_num == 0: 
                    keys = line.strip().split()

                    for k in keys:
                        score_matrix[k]= {}
                
                # Last line of matrix contains the gap penalty which must be pulled
                # out and returned.
                elif len(line.strip().split())== 1:
                    gap_penalty = int(line.strip())

                else:
                    values = [int(h) for h in line.strip().split()]
                    val= keys[line_num - 1]
                    # list comprehension: say what you want it to first 
                    score_matrix[val]= dict(zip(keys, values))
                

        return score_matrix, gap_penalty

    @staticmethod
    def load_FASTA(fname):
        """
        Input: (String) A path to a FASTA file containing exactly two sequences.
        Output: (List) A list containing two strings: one for each sequence.

        Example:

        >>> seqs = NWAligner.load_FASTA('example.fa')
        >>> seqs[0]
        'YAADSKATPGNPAFHQDEIFLARIAFIYQMWDGGQLKLIDYAPHHVMCEE'
        >>> seqs[1]
        'WVGQPNMKVQHWSNMKACCVKFITWTFIAPEKHACKWTETAYQADCDIIW'
        >>> len(seqs)
        2

        """

        seqs = []
        
        # Load FASTA file and return list of sequences.
        with open(fname) as fp:
            for line_num, line in enumerate(fp): 
                if line_num <= 3:
                    if line_num%2 > 0:
                        seqs.append(line.strip())
                
                # Throw an error if there are more than two sequences in the file.
                else: 
                    if line_num > 3:
                        sys.exit("More than 2 seqences provided")

        return seqs

    def align(self, seq_x, seq_y, print_matrix = False):
        """
        Input: (Strings) Two sequences to be aligned (seq_x and seq_y).
               (Boolean) If print_matrix is True, print the dynamic programming
                         matrix before traceback.
        Output: (Tuple of strings) Two sequences, aligned.

        Example:

        >>> aligner = NWAligner('BLOSUM62')
        >>> seqs = aligner.load_FASTA('example.fa')
        >>> aligner.align(seqs[0], seqs[1])
        ('YAAD-SKATPGNPAF---HQDEIF--L-AR--IA-FIYQM-WDGGQLK-LIDYAPH-HVM-C---E-------E---',
         'W---VGQ--P-N--MKVQH----WSNMKA-CCV-KFI---TW------TFI--APEKH--ACKWTETAYQADCDIIW')

        """

        ###
        ### INITIALIZATION
        ###

        # create two empty matrices with sizes based on the input sequences.
        # one contains the dynamic programming matrix, the other contains
        # pointers we'll use during traceback
        matrix = [[0] * (len(seq_y) + 1) for _ in range(len(seq_x) + 1)]
        pointers = [[0] * (len(seq_y) + 1) for _ in range(len(seq_x) + 1)]

        
        # Fill the top row and first column of the matrix with scores for gaps.
        
        for x in range(len(seq_x)):
            matrix[0][x] = self.gap_penalty*(x)
                
        for y in range(len(seq_y)):
            matrix[y][0] = self.gap_penalty*(y)
        
        ###
        ### RECURSION
        ###

        # fill the dynamic programming and pointer matrices
        for x in range(1, len(seq_x) + 1):
            for y in range(1, len(seq_y) + 1):
                match_score = self.score_matrix[seq_x[x - 1]][seq_y[y - 1]]

                pointers[0][y] = 'h'
                pointers[x][0] = 'v' 

                if x and y > 0:
                    diag = matrix[x-1][y-1] + match_score
                    h_gap = matrix[x-1][y] + self.gap_penalty
                    v_gap = matrix[x][y-1] + self.gap_penalty
                    matrix[x][y] = max(diag, h_gap, v_gap)

                    if matrix[x][y] == diag: 
                        pointers[x][y]= 'd'
                    elif matrix[x][y] == h_gap:
                        pointers[x][y]= 'h'
                    elif matrix[x][y] == v_gap:
                        pointers[x][y]= 'v'

            if  matrix[1][1] > self.gap_penalty:
                pointers[0][0] = 'd'

        # print the dynamic programming matrix
        if print_matrix:
            for x in range(len(seq_x) + 1):
                print " ".join(map(lambda i: str(int(i)), matrix[x]))

        ###
        ### TRACEBACK
        ###

        # starting from the bottom right corner, follow the pointers back
        x, y = len(seq_x), len(seq_y)

        # fill these lists with the aligned sequences
        align_x = []
        align_y = []

        while x > 0 or y > 0:
            move = pointers[x][y]
            
       # depending on the pointer the next move will be made and seqs aligned accordingly
            if move == "d":
                x = x-1
                y = y-1
            
                align_x.append(seq_x[x])
                align_y.append(seq_y[y])
                
                 
            if move == "h":
                x = x-1
                y = y
            
                align_x.append(seq_x[x])
                align_y.append("-")

            if move == "v":
                x = x
                y = y-1
            
                align_x.append("-")
                align_y.append(seq_y[y])    
            

        # flip the alignments, as they're reversed
        return ("".join(align_x[::-1]), "".join(align_y[::-1]))

###                                      ###
### NO NEED TO EDIT CODE BELOW THIS LINE ###
###                                      ###

if __name__ == '__main__':
    def usage():
        print 'usage: %s matrixfilename stringfilename'
        sys.exit(1)

    if len(sys.argv) != 3:
        usage()

    for fname in sys.argv[1:]:
        if not os.path.isfile(fname):
            print 'Can not open %s' % (fname,)
            usage()

    aligner = NWAligner(sys.argv[1])
    seqs = aligner.load_FASTA(sys.argv[2])
    result = aligner.align(seqs[0], seqs[1])

    print('>seq1\n%s\n>seq2\n%s' % (result[0], result[1]))

SyntaxError: invalid syntax (<ipython-input-1-df98d2f6b108>, line 171)

## Test Run: 

In [4]:
import os
import sys

class NWAligner:
    def __init__(self, score_matrix_fname):
        self.score_matrix, self.gap_penalty = self.load_score_matrix(score_matrix_fname)

    @staticmethod
    def load_score_matrix(fname):
        
        score_matrix = {}
        gap_penalty = None
        keys = []

        with open(fname) as fp:
            for line_num, line in enumerate(fp):
                # ignore comments in matrix file
                if line.startswith("#"):
                    continue
                # Parse matrix file line-by-line and load into nested dictionaries.
                elif line_num == 0: 
                    keys = line.strip().split()

                    for k in keys:
                        score_matrix[k]= {}
                
                # Last line of matrix contains the gap penalty which must be pulled
                # out and returned.
                elif len(line.strip().split())== 1:
                    gap_penalty = int(line.strip())

                else:
                    values = [int(h) for h in line.strip().split()]
                    val= keys[line_num - 1]
                    # list comprehension: say what you want it to first 
                    score_matrix[val]= dict(zip(keys, values))
                

        return score_matrix, gap_penalty

    @staticmethod
    def load_FASTA(fname):
        
        seqs = []
        
        # Load FASTA file and return list of sequences.
        with open(fname) as fp:
            for line_num, line in enumerate(fp): 
                if line_num <= 3:
                    if line_num%2 > 0:
                        seqs.append(line.strip())
                
                # Throw an error if there are more than two sequences in the file.
                else: 
                    if line_num > 3:
                        sys.exit("More than 2 seqences provided")

        return seqs

    def align(self, seq_x, seq_y, print_matrix = False):
        
        ###
        ### INITIALIZATION
        ###

        # create two empty matrices with sizes based on the input sequences.
        # one contains the dynamic programming matrix, the other contains
        # pointers we'll use during traceback
        matrix = [[0] * (len(seq_y) + 1) for _ in range(len(seq_x) + 1)]
        pointers = [[0] * (len(seq_y) + 1) for _ in range(len(seq_x) + 1)]

        
        # Fill the top row and first column of the matrix with scores for gaps.
        
        for x in range(len(seq_x)):
            matrix[0][x] = self.gap_penalty*(x)
                
        for y in range(len(seq_y)):
            matrix[y][0] = self.gap_penalty*(y)
        
        ###
        ### RECURSION
        ###

        # fill the dynamic programming and pointer matrices
        for x in range(1, len(seq_x) + 1):
            for y in range(1, len(seq_y) + 1):
                match_score = self.score_matrix[seq_x[x - 1]][seq_y[y - 1]]

                pointers[0][y] = 'h'
                pointers[x][0] = 'v' 

                if x and y > 0:
                    diag = matrix[x-1][y-1] + match_score
                    h_gap = matrix[x-1][y] + self.gap_penalty
                    v_gap = matrix[x][y-1] + self.gap_penalty
                    matrix[x][y] = max(diag, h_gap, v_gap)

                    if matrix[x][y] == diag: 
                        pointers[x][y]= 'd'
                    elif matrix[x][y] == h_gap:
                        pointers[x][y]= 'h'
                    elif matrix[x][y] == v_gap:
                        pointers[x][y]= 'v'

            if  matrix[1][1] > self.gap_penalty:
                pointers[0][0] = 'd'

        # print the dynamic programming matrix
        if print_matrix:
            for x in range(len(seq_x) + 1):
                print(" ".join(map(lambda i: str(int(i)), matrix[x])))

        ###
        ### TRACEBACK
        ###

        # starting from the bottom right corner, follow the pointers back
        x, y = len(seq_x), len(seq_y)

        # fill these lists with the aligned sequences
        align_x = []
        align_y = []

        while x > 0 or y > 0:
            move = pointers[x][y]
            
       # depending on the pointer the next move will be made and seqs aligned accordingly
            if move == "d":
                x = x-1
                y = y-1
            
                align_x.append(seq_x[x])
                align_y.append(seq_y[y])
                
                 
            if move == "h":
                x = x-1
                y = y
            
                align_x.append(seq_x[x])
                align_y.append("-")

            if move == "v":
                x = x
                y = y-1
            
                align_x.append("-")
                align_y.append(seq_y[y])    
            

        # flip the alignments, as they're reversed
        return ("".join(align_x[::-1]), "".join(align_y[::-1]))

aligner = NWAligner('BLOSUM62')
seqs = aligner.load_FASTA('example.fa')
aligner.align(seqs[0], seqs[1], True)

0 -4 -8 -12 -16 -20 -24 -28 -32 -36 -40 -44 -48 -52 -56 -60 -64 -68 -72 -76 -80 -84 -88 -92 -96 -100 -104 -108 -112 -116 -120 -124 -128 -132 -136 -140 -144 -148 -152 -156 -160 -164 -168 -172 -176 -180 -184 -188 -192 -196 0
-4 2 -2 -6 -10 -14 -18 -22 -26 -30 -34 -38 -42 -46 -50 -54 -58 -62 -66 -70 -74 -78 -81 -85 -89 -93 -97 -101 -105 -109 -113 -117 -121 -125 -129 -133 -137 -141 -145 -149 -153 -157 -157 -161 -165 -169 -173 -177 -181 -185 -4
-8 -2 2 -2 -6 -10 -14 -18 -22 -26 -30 -34 -38 -41 -45 -49 -53 -54 -58 -62 -66 -70 -74 -78 -82 -86 -90 -94 -98 -101 -105 -109 -113 -117 -121 -125 -129 -133 -137 -141 -145 -149 -153 -157 -157 -161 -165 -169 -173 -177 -8
-12 -6 -2 2 -2 -6 -10 -14 -18 -22 -26 -30 -34 -37 -41 -45 -49 -49 -53 -57 -61 -65 -69 -73 -77 -81 -85 -89 -93 -94 -98 -102 -106 -110 -113 -117 -121 -125 -129 -133 -137 -141 -145 -149 -153 -157 -161 -165 -169 -173 -12
-16 -10 -6 -2 2 -2 -5 -9 -13 -17 -21 -25 -29 -33 -36 -40 -44 -48 -52 -56 -60 -62 -66 -70 -74 -78 -82 -86 -90 -94 -95 -96 

('YAADSKATPGNPAFHQDEIFLARIAFI-YQ-MW-DGGQLKLIDYAPHHVMCE--E',
 'W-V-GQ--PNMKVQHWSNMKACCVKFITWTFIAPEKHACKWTETA-YQADCDIIW')